Here I make use of BART, a state-of-the-art transformer model that was pretrained on a CNN news corpus. I am mostly making use of the HuggingFace transformers package, without much modification.

## Defining BART model for summarization

In [1]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [2]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [3]:
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu '# failing when device is gpu
torch_device = 'cpu'

In [4]:
def bart_summarize(text, num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size):
  
    text = text.replace('\n','')
    text_input_ids = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
    summary_ids = model.generate(text_input_ids, num_beams=int(num_beams), length_penalty=float(length_penalty), max_length=int(max_length), min_length=int(min_length), no_repeat_ngram_size=int(no_repeat_ngram_size))           
    summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    return summary_txt


### test

In [5]:
test = False

In [6]:
text = "A longstanding problem for Deep Neural Networks (DNNs) is understanding their puzzling ability to generalize well. We approach this problem through the unconventional angle of \textit{cognitive abstraction mechanisms}, drawing inspiration from recent neuroscience work, allowing us to define the Cognitive Neural Activation metric (CNA) for DNNs, which is the correlation between information complexity (entropy) of given input and the concentration of higher activation values in deeper layers of the network. The CNA is highly predictive of generalization ability, outperforming norm-and-margin-based generalization metrics on an extensive evaluation of over 100 dataset-and-network-architecture combinations, especially in cases where additive noise is present and/or training labels are corrupted. These strong empirical results show the usefulness of CNA as a generalization metric, and encourage further research on the connection between information complexity and representations in the deeper layers of networks in order to better understand the generalization capabilities of DNNs."

In [7]:
if test:
    bart_summarize(text = text, num_beams = 4, length_penalty = 2.0, max_length = 142, min_length = 56, no_repeat_ngram_size = 3)

## With actual transcripts:

In [10]:
load_summaries = False # set to true if you want to compute the summaries here rather than loading an output

In [11]:
import pandas as pd

In [13]:
data = pd.read_csv('full_data/train.csv')

In [14]:
data.shape

(32626, 3)

In [ ]:
import time
N = data.shape[0]
if not load_summaries:
    bartcnn_summaries = []
    t1 = time.time()
    for k, transcript in enumerate(data['transcript']):
        if k > N:
            break
        
        mod = 5
        if k%mod == 0:
            print('On transcript...',k)
            if k != 0:
                t2 = time.time()
                print('ETA:', (t2-t1)/60 * (N - k) / mod, 'minutes')
                t1 = time.time()
            
        bartcnn_summaries.append(bart_summarize(text = transcript, num_beams = 4, length_penalty = 2.0, max_length = 142, min_length = 56, no_repeat_ngram_size = 3))

In [ ]:
if not load_summaries:
    data_woutput = data.iloc[:N+1].copy()
    data_woutput['bartcnn_summaries'] = bartcnn_summaries

In [ ]:
data_woutput.to_csv('full_data/data_woutput.csv', index = False)

In [7]:
if load_summaries:
    data_woutput = pd.read_csv('processed_data/data_woutput.csv')

In [8]:
data_woutput.head()

,transcript,episode_description,bartcnn_summaries
0,Whatever happened in dealing with anxiety didn...,"Thank you. In this episode, Alec talks abou...",Grief is one of the few things that are consta...
1,Free to fly and step up and step podcast the p...,We're back for another week with Missed Connec...,A hospital worker at Gulf Coast hospital was a...
2,A dog that walks on its hind legs a monster th...,This episode is brought to you by Haunted Plac...,This episode is part of our urban legends Hall...
3,"Well, hey there Riley. Hey Jack. How's it goin...",Welcome ladies and gentleman to the brand new ...,Riley espresso Parris and Jack Sterling are th...
4,"Fitness mantra Hello, those two guess haplogr...",Carpal Tunnel Syndrome. What is it? Reasons du...,Fitness mantra. Met Javi our job subsea up. Ne...


Rouge scores of bart non-fine tuned model, on the full set:

{'rouge-1': {'r': 0.22204752142115872,
  'p': 0.25994254225911884,
  'f': 0.22304858048809925},
 'rouge-2': {'r': 0.047035517328465455,
  'p': 0.05944517202472876,
  'f': 0.04742784885138758},
 'rouge-l': {'r': 0.19345402530460704,
  'p': 0.22843380011356698,
  'f': 0.19501811621677081}}